# 4.1.2 Running Prednet on Full I3D Data

The previous notebook was only running on some of the I3D data. This one will use all the data.

## Jupyter Extensions

Load [watermark](https://github.com/rasbt/watermark) to see the state of the machine and environment that's running the notebook. To make sense of the options, take a look at the [usage](https://github.com/rasbt/watermark#usage) section of the readme.

In [1]:
# Load `watermark` extension
%load_ext watermark
# Display the status of the machine and packages. Add more as necessary.
%watermark -v -n -m -g -b -t -p torch,torchvision,cv2,h5py,pandas,matplotlib,seaborn,jupyterlab,lab

Fri Mar 06 2020 17:26:58 

CPython 3.6.10
IPython 7.12.0

torch 1.2.0
torchvision 0.1.8
cv2 3.4.2
h5py 2.8.0
pandas 1.0.1
matplotlib 3.1.3
seaborn 0.10.0
jupyterlab 1.2.6
lab 0+untagged.38.g6a19aca.dirty

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-76-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 16
interpreter: 64bit
Git hash   : 6a19aca9e16ed91aaf852f0914eb45b18ea68d92
Git branch : master


Load [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) which will always reload modules marked with `%aimport`.

This behavior can be inverted by running `autoreload 2` which will set everything to be auto-reloaded *except* for modules marked with `%aimport`.

In [2]:
# Load `autoreload` extension
%load_ext autoreload
# Set autoreload behavior
%autoreload 1

Load `matplotlib` in one of the more `jupyter`-friendly [rich-output modes](https://ipython.readthedocs.io/en/stable/interactive/plotting.html). Some options (that may or may not have worked) are `inline`, `notebook`, and `gtk`.

In [3]:
# Set the matplotlib mode
%matplotlib inline

## Set the GPU

Make sure we aren't greedy.

In [4]:
!nvidia-smi

Fri Mar  6 17:27:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:04:00.0 Off |                  N/A |
| 28%   48C    P2    69W / 250W |   9399MiB / 12196MiB |     31%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:05:00.0 Off |                  N/A |
| 39%   64C    P2   126W / 250W |  12037MiB / 12196MiB |     40%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [5]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


## Imports

In [6]:
from pathlib import Path

import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

Local imports that may or may not be autoreloaded. This section contains things that will likely have to be re-imported multiple times, and have additions or subtractions made throughout the project.

In [7]:
# Constants to be used throughout the package
%aimport lab
import lab
%aimport lab.index
from lab.index import DIR_DATA_INT, DIR_DATA_RAW
%aimport lab.breakfast
import lab.breakfast as bk
%aimport lab.breakfast.constants
from lab.breakfast.constants import SEED
# Import the data subdirectories
%aimport lab.breakfast.index
from lab.breakfast.index import (DIR_BREAKFAST, 
                                 DIR_BREAKFAST_DATA, 
                                 DIR_COARSE_SEG, 
                                 DIR_FINE_SEG,
                                 DIR_BK_WEIGHTS,
                                 DIR_BK_CHECKPOINTS,
                                )
%aimport lab.breakfast.prednet
from lab.breakfast.prednet import PredNet
%aimport lab.breakfast.dataloader
from lab.breakfast.dataloader import Breakfast64DimFVDataset, BreakfastI3DFVDataset

## Initial Setup

Set [seaborn defaults](https://seaborn.pydata.org/generated/seaborn.set.html) for matplotlib.

In [8]:
sns.set()

## Running with I3D Data

### DataLoader et al

Loading the Dataloader which now has all the I3D data.

In [10]:
%%time
ds = BreakfastI3DFVDataset()

CPU times: user 10.8 s, sys: 2min 7s, total: 2min 18s
Wall time: 8min 10s


In [18]:
np.random.seed(SEED)

ds_length = len(ds)
indices = list(range(ds_length))
batch_size = 256
n_test = np.maximum(batch_size, 128)

np.random.shuffle(indices)
train_indices, test_indices = indices[n_test:], indices[:n_test]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(ds, batch_size=batch_size, sampler=train_sampler)
test_loader = DataLoader(ds, batch_size=batch_size, sampler=test_sampler)

In [19]:
del model, layer_loss_weights, time_loss_weights
torch.cuda.empty_cache()

### Running the Model

In [23]:
%%time

num_epochs = 50
n_layers = 4
input_size = 2048
nt = 64 # num of time steps
A_channels = tuple(input_size // (2**i) for i in range(n_layers))
R_channels = tuple(input_size // (2**i) for i in range(n_layers))
lr = 0.000333 # if epoch < 75 else 0.0001

path_checkpoint = DIR_BK_CHECKPOINTS / 'i3d_checkpoint.tar'
path_weights = DIR_BK_WEIGHTS / 'i3d_training.pt'

layer_loss_weights = Variable(torch.FloatTensor([[1.]] + [[0.]]*(n_layers-1)).cuda())
time_loss_weights = 1./(nt - 1) * torch.ones(nt, 1)
time_loss_weights[0] = 0
time_loss_weights = Variable(time_loss_weights.cuda())

model = PredNet(R_channels, A_channels, output_mode='error')
print(model)
if torch.cuda.is_available():
    print('Using GPU.')
    model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def lr_scheduler(optimizer, epoch):
    if epoch < num_epochs // 2:
        return optimizer
    else:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001
        return optimizer
    
train_errors = []

print(f'Running with batch size {batch_size} ({ds_length//batch_size} iterations / epoch)')
    
for epoch in range(num_epochs):
    optimizer = lr_scheduler(optimizer, epoch)
    for batch_idx, (data, path) in enumerate(train_loader):
        data = Variable(data)
        errors = self.forward(data) # batch x n_layers x nt
        loc_batch = errors.size(0)
        errors = torch.mm(errors.view(-1, nt), self.time_loss_weights) # batch*n_layers x 1
        errors = torch.mm(errors.view(loc_batch, -1), self.layer_loss_weights)
        errors = torch.mean(errors, axis=0)
        train_errors.append(errors.item())

        optimizer.zero_grad()
        errors.backward()
        optimizer.step()

    if epoch % 2 == 0:
        test_errors = []
        for data, path in test_loader:
            data = Variable(data)
            errors = model(data) # batch x n_layers x nt
            loc_batch = errors.size(0)
            errors = torch.mm(errors.view(-1, nt), time_loss_weights) # batch*n_layers x 1
            errors = torch.mm(errors.view(loc_batch, -1), layer_loss_weights)
            test_errors.append(torch.mean(errors, axis=0).item())
            
        test_error = np.mean(test_errors)
        train_error = np.mean(train_errors)
        print(f'Epoch: {epoch}/{num_epochs}, train_error: {train_error}, '
              f'test error: {test_error}')
        train_errors = []
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'errors': errors,
        }, str(path_checkpoint))


PredNet(
  (cell0): LSTM(
    (i2h): Linear(in_features=4096, out_features=8192, bias=True)
    (h2h): Linear(in_features=2048, out_features=8192, bias=True)
  )
  (cell1): LSTM(
    (i2h): Linear(in_features=2048, out_features=4096, bias=True)
    (h2h): Linear(in_features=1024, out_features=4096, bias=True)
  )
  (cell2): LSTM(
    (i2h): Linear(in_features=1024, out_features=2048, bias=True)
    (h2h): Linear(in_features=512, out_features=2048, bias=True)
  )
  (cell3): LSTM(
    (i2h): Linear(in_features=512, out_features=1024, bias=True)
    (h2h): Linear(in_features=256, out_features=1024, bias=True)
  )
  (dense0): Sequential(
    (0): Linear(in_features=2048, out_features=2048, bias=True)
    (1): ReLU()
    (satlu): SatLU (min_val=0, max_val=255)
  )
  (dense1): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): ReLU()
  )
  (dense2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU()
  )
  (dense3): Sequ

In [24]:
torch.save(model.state_dict(), str(path_weights))